In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession # https://spark.apache.org/docs/1.6.1/sql-programming-guide.html
from pyspark.sql import functions as F

In [3]:
warehouse_location = os.path.abspath('spark-warehouse/test')
# Create the session
conf = SparkConf() \
    .set("spark.ui.port", "4041") \
    .set('spark.sql.warehouse.dir', warehouse_location)

# Create the context
sc = SparkContext(conf=conf)
spark = SparkSession.builder \
    .appName('Spark451') \
    .config("hive.metastore.uris", "thrift://localhost:9083") \
    .enableHiveSupport() \
    .getOrCreate()

sqlContext = SQLContext(sc)

In [4]:
%%time
# https://stackoverflow.com/questions/56927329/spark-option-inferschema-vs-header-true/56933052
df_master = (spark.read.format('csv')
    .option('header', 'true') # first row is column names.
    .option("inferSchema", "true") # set column types       
    .load('../data/irs990/master_file/*.csv'))

CPU times: user 5.37 ms, sys: 207 µs, total: 5.57 ms
Wall time: 9.93 s


In [8]:
df_master = df_master.toDF(*[c.lower() for c in df_master.columns]) # create lowercase columns
df_master.createOrReplaceTempView("master")

In [9]:
%%time
df_master.count()

CPU times: user 1.46 ms, sys: 202 µs, total: 1.67 ms
Wall time: 1.04 s


1757712

In [18]:
%%time
spark.sql('SELECT sum(revenue_amt) as sum from master').toPandas()

CPU times: user 7.92 ms, sys: 1.11 ms, total: 9.03 ms
Wall time: 1.2 s


,sum
0,2932332292337


In [11]:
df_master.groupBy('state').count().show()

+-----+------+
|state| count|
+-----+------+
|   SC| 28104|
|   AZ| 26683|
|   LA| 21598|
|   MN| 36629|
|   AA|     1|
|   NJ| 47542|
|   DC| 13685|
|   OR| 25038|
|   VA| 47858|
| null|  1291|
|   RI|  8457|
|   KY| 20781|
|   WY|  5095|
|   NH|  8776|
|   MI| 52331|
|   NV| 13519|
|   WI| 37093|
|   ID|  9227|
|   CA|180569|
|   NE| 14115|
+-----+------+
only showing top 20 rows

